# Getting solution information

In this tutorial we will see how to extract information from an optimization model, such as solution information. Gurobi has organized this information in so called *attributes*. Each component of an optimization model, i.e., the [Model object](https://www.gurobi.com/documentation/9.0/refman/py_model.html), the [variables objects](https://www.gurobi.com/documentation/9.0/refman/py_var.html) and the [constraints objects](https://www.gurobi.com/documentation/9.0/refman/py_constr.html) are provided with a method called `getAttr` which can be used to query their specific attributes, corresponding to solution and model information. A central exercise is thus that of understanding which attributes can be queried from each object. The full list of attributes for each object is provided in the Gurobi documentation [here](https://www.gurobi.com/documentation/9.0/refman/attributes.html#sec:Attributes). This page should be kept at hand when working with optimization models.

## An example model

In this tutorial we will consider the classical diet problem. The miminum cost diet is to be found in order to satisfy the daily requirement $R_n$ of a set $\mathcal{N}$ of nutrients such as proteins and vitamins. A set $\mathcal{F}$ of foods are considered. Each food has a cost of $C_f$ per unit (e.g., per gram) and provides an amount $A_{fn}$ of nutrient $n$. Let $x_f$ be the quantity of food $f$ included in the daily diet. The problem can be formulated as follows
$$\min \sum_{f\in\mathcal{F}}C_{f}x_{f}$$
$$\sum_{f\in\mathcal{F}}A_{fn}x_{f} = R_n \qquad \forall n \in\mathcal{N}$$
$$x_{f}\geq 0 \qquad \forall f\in\mathcal{F}$$


In [9]:
from gurobipy import *
# Data
foods = ['apples', 'bananas', 'carrots', 'dates','eggs']
nutrients = ['proteins','vitamin_c','iron']
cost = {'apples':8, 'bananas':10, 'carrots':3, 'dates':20,'eggs':15}
composition = {('apples','proteins'):0.4, 
               ('apples','vitamin_c'):6, 
               ('apples','iron'):0.4, 
               ('bananas','proteins'):1.2, 
               ('bananas','vitamin_c'):10, 
               ('bananas','iron'):0.6,
               ('carrots','proteins'):0.6, 
               ('carrots','vitamin_c'):3, 
               ('carrots','iron'):0.4,
               ('dates','proteins'):0.6, 
               ('dates','vitamin_c'):1, 
               ('dates','iron'):0.2,
               ('eggs','proteins'):12.2, 
               ('eggs','vitamin_c'):0, 
               ('eggs','iron'):2.6
              }
requirements = {'proteins':70,'vitamin_c':60,'iron':12}
# Model
m = Model('diet_problem')
x = m.addVars(foods,name = 'x')
expr = x.prod(cost)
m.setObjective(expr,GRB.MINIMIZE)
m.addConstrs((quicksum([composition[f,n] * x[f] for f in foods]) >= requirements[n] for n in nutrients), 'nutri')
m.optimize()
m.write("diet.lp")
print('Objective value: %g' % m.objVal)
for f in foods:
    print('%s %g' % (x[f].varName, x[f].x))

Optimize a model with 3 rows, 5 columns and 14 nonzeros
Coefficient statistics:
  Matrix range     [2e-01, 1e+01]
  Objective range  [3e+00, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 7e+01]
Presolve removed 0 rows and 1 columns
Presolve time: 0.01s
Presolved: 3 rows, 4 columns, 11 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.825000e+01   0.000000e+00      0s
       3    1.3131148e+02   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.02 seconds
Optimal objective  1.313114754e+02
Objective value: 131.311
x[apples] 0
x[bananas] 0
x[carrots] 20
x[dates] 0
x[eggs] 4.7541


## Attributes of the Model

Let us start by having a look at some of the most important attributes of the `Model` object.
First of all, in the documentation you can see that the `Model` object has a [method `getAttr`](https://www.gurobi.com/documentation/9.0/refman/py_model_getattr.html). This method can be used to query information about the model as well as information about sets of decision variables and constraints. Let us see some examples.